# **Proyek Predictive Analytics: Prediksi Penyakit Jantung**

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

## **1. Mengimpor Library yang Dibutuhkan**
Langkah pertama adalah mengimpor semua library Python yang akan digunakan.

In [ ]:
# Mengabaikan peringatan yang tidak relevan agar output lebih bersih
import warnings
warnings.filterwarnings('ignore')

# Library untuk manipulasi dan analisis data
import pandas as pd
import numpy as np

# Library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns

# Library dari Scikit-learn untuk machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Mengatur agar plot ditampilkan inline di notebook
%matplotlib inline

## **2. Data Loading dan Understanding**
Pada tahap ini, kita akan memuat dataset, yaitu "Heart Failure Prediction" dari file CSV yang diunggah. Dataset ini memiliki 918 sampel.

In [ ]:
# Memuat dataset dari file CSV
# Pastikan file 'heart.csv' berada di direktori yang sama atau unggah ke environment Colab
df = pd.read_csv('heart.csv')

# Menampilkan 5 baris pertama dari dataframe
print("Data Awal (5 baris pertama):")
display(df.head())

# Menampilkan informasi dasar dataframe untuk memahami tipe data dan non-null counts
print("\nInformasi DataFrame:")
df.info()

# Variabel target kita adalah 'HeartDisease' (1: Gagal Jantung, 0: Sehat)

## **3. Exploratory Data Analysis (EDA)**
Melakukan analisis dan visualisasi pada data untuk mendapatkan wawasan lebih dalam. Ini termasuk memeriksa distribusi fitur, keseimbangan kelas target, dan korelasi antar fitur.

In [ ]:
# Menampilkan statistik deskriptif untuk fitur numerik
print("Statistik Deskriptif:")
display(df.describe())

# Memisahkan fitur numerik dan kategorikal untuk analisis lebih lanjut
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Variabel target 'HeartDisease' adalah numerik (0 atau 1) jadi kita pindahkan dari categorical
numerical_cols.remove('HeartDisease')

print(f"\nFitur Numerik: {numerical_cols}")
print(f"Fitur Kategorikal: {categorical_cols}")

In [ ]:
# Visualisasi distribusi variabel target
plt.figure(figsize=(6, 4))
sns.countplot(x='HeartDisease', data=df)
plt.title('Distribusi Variabel Target (0: Sehat, 1: Gagal Jantung)')
plt.xlabel('Status Gagal Jantung')
plt.ylabel('Jumlah Pasien')
plt.show()
print(df['HeartDisease'].value_counts())

# Visualisasi distribusi fitur numerik
df[numerical_cols].hist(figsize=(14, 10), bins=20)
plt.suptitle('Distribusi Fitur Numerik')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# Visualisasi distribusi fitur kategorikal
plt.figure(figsize=(14, 12))
for i, col in enumerate(categorical_cols):
    plt.subplot(3, 2, i + 1)
    sns.countplot(x=col, data=df, hue='HeartDisease')
    plt.title(f'Distribusi {col} berdasarkan Gagal Jantung')
    plt.xlabel('')
    plt.ylabel('Jumlah')
plt.tight_layout()
plt.show()

## **4. Data Cleaning & Preparation**

Tahap ini meliputi:
1. Pemisahan fitur dan target.
2. Penanganan data duplikat.
3. Pemisahan data menjadi set pelatihan dan pengujian.
4. Pembuatan pipeline untuk preprocessing (standarisasi untuk numerik dan one-hot encoding untuk kategorikal).

Catatan: Dataset ini tidak memiliki missing values, namun kita akan tetap memeriksa dan menangani data duplikat. Penanganan outlier tidak dilakukan karena distribusi data numerik cukup wajar dan dalam konteks medis, nilai ekstrem bisa jadi data valid.

In [ ]:
# Memisahkan fitur (X) dan target (y)
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

# Memeriksa dan menangani data duplikat
print(f"Jumlah data duplikat sebelum dihapus: {X.duplicated().sum()}")
if X.duplicated().sum() > 0:
    # Simpan indeks data yang tidak duplikat
    non_duplicate_indices = ~X.duplicated()
    X = X[non_duplicate_indices]
    y = y[non_duplicate_indices]
    print(f"Jumlah data duplikat setelah dihapus: {X.duplicated().sum()}")

print(f"\nUkuran data setelah dibersihkan: {X.shape}")

# Membagi data menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nUkuran data latih: {X_train.shape}")
print(f"Ukuran data uji: {X_test.shape}")

In [ ]:
# Membuat pipeline untuk preprocessing
# Fitur numerik akan di-scale menggunakan StandardScaler
# Fitur kategorikal akan di-encode menggunakan OneHotEncoder

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Membuat preprocessor dengan ColumnTransformer
# Ini akan menerapkan transformer yang berbeda ke kolom yang berbeda
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='passthrough' # Kolom yang tidak disebutkan akan dilewatkan
)

# Menampilkan pipeline preprocessor
print("Pipeline Preprocessing:")
display(preprocessor)

## **5. Modeling**
Membangun, melatih, dan mengevaluasi dua model klasifikasi: Logistic Regression dan Random Forest. Akan mengintegrasikan preprocessor ke dalam pipeline model untuk memastikan data diproses dengan benar sebelum pelatihan.

In [ ]:
# === Model 1: Logistic Regression ===

# Membuat pipeline lengkap yang menggabungkan preprocessor dan model
log_reg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

# Melatih model Logistic Regression menggunakan pipeline
print("Melatih model Logistic Regression...")
log_reg_pipeline.fit(X_train, y_train)
print("Model Logistic Regression berhasil dilatih.\n")


# === Model 2: Random Forest ===

# Membuat pipeline lengkap untuk Random Forest
rand_forest_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42))
])

# Melatih model Random Forest menggunakan pipeline
print("Melatih model Random Forest...")
rand_forest_pipeline.fit(X_train, y_train)
print("Model Random Forest berhasil dilatih.")

## **6. Evaluation**
Evaluasi performa kedua model menggunakan berbagai metrik pada data uji. Fungsi evaluasi yang sama akan digunakan untuk memastikan perbandingan yang adil.

In [ ]:
# Fungsi untuk mengevaluasi model dan menampilkan hasilnya
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)

    # Menghitung metrik evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Menampilkan hasil
    print(f"--- Evaluasi Model: {model_name} ---")
    print(f"Akurasi: {accuracy:.3f}")
    print(f"Presisi: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1-Score: {f1:.3f}\n")
    
    # Menampilkan Laporan Klasifikasi
    print("Laporan Klasifikasi:")
    print(classification_report(y_test, y_pred, target_names=['Sehat', 'Gagal Jantung']))
    
    # Menampilkan Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Sehat', 'Gagal Jantung'],
                yticklabels=['Sehat', 'Gagal Jantung'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Prediksi')
    plt.ylabel('Aktual')
    plt.show()
    
    return {'Akurasi': accuracy, 'Presisi': precision, 'Recall': recall, 'F1-Score': f1}

# Mengevaluasi kedua model pipeline
log_reg_metrics = evaluate_model(log_reg_pipeline, X_test, y_test, "Logistic Regression")
rand_forest_metrics = evaluate_model(rand_forest_pipeline, X_test, y_test, "Random Forest")

## **7. Perbandingan Hasil dan Kesimpulan**
Membandingkan metrik evaluasi dari kedua model untuk menentukan model mana yang memberikan performa terbaik untuk kasus prediksi gagal jantung.

In [ ]:
# Membuat DataFrame untuk membandingkan metrik dari kedua model
metrics_df = pd.DataFrame({
    'Logistic Regression': log_reg_metrics,
    'Random Forest': rand_forest_metrics
}).T # .T untuk transpose (menukar baris dan kolom)

print("Perbandingan Metrik Evaluasi:")
display(metrics_df.round(3))

# Menentukan model terbaik berdasarkan F1-Score
best_model_name = metrics_df['F1-Score'].idxmax()
print(f"\nKesimpulan: Model terbaik berdasarkan F1-Score adalah **{best_model_name}**.")